In [ ]:
import os
import pickle
import models

import numpy as np
import numpy.ma as ma
import matplotlib.pyplot as plt
import seaborn as sns
from numpyro.diagnostics import hpdi

from hbmep.nn import functional as F
from hbmep.model.utils import Site as site

from hbmep.config import Config
from hbmep.model.utils import Site as site
from scipy import stats

from models import NonHierarchicalBayesianModel

USER = os.environ["USER"]

MAX_THRESHOLD_CONST = 3.50 #CONSTANT WILL BE mean of rats for stim max / threshold for EVERY possible combination... ma.mean(ma.median((stim_max/ma.mean(a, axis=0)),axis = (1,2,3)),axis =0)
NUM_THRESHOLD_POINTS = 500


In [ ]:
src = "/home/andres/repos/rat-mapping-paper/reports/L_CIRC/inference.pkl"

with open(src, "rb") as f:
    df, encoder_dict, model, posterior_samples, = pickle.load(f);

In [5]:
named_params = [site.a, site.b, site.L, site.ell, site.H]
params = [posterior_samples[param][...] for param in named_params]

for named_param, param in zip(named_params, params):
    print(named_param, param.shape)

a (4000, 8, 21, 6)
b (4000, 8, 21, 6)
L (4000, 8, 21, 6)
ℓ (4000, 8, 21, 6)
H (4000, 8, 21, 6)


In [13]:
compound_positions = encoder_dict["compound_position"].inverse_transform(sorted(set(df["compound_position"]))).tolist() #sorted(set(df["compound_position"])

In [15]:
vertices = [(i, cpos) for i, cpos in enumerate(compound_positions) if cpos.split("-")[0] == ""]
radii = [(i, cpos) for i, cpos in enumerate(compound_positions) if cpos.split("-")[0] and cpos.split("-")[1] == "C6LC"]
diameters = [(i, cpos) for i, cpos in enumerate(compound_positions) if (i, cpos) not in vertices and (i, cpos) not in radii]

vertices = [i for (i, cpos) in vertices]
radii = [i for (i, cpos) in radii]
diameters = [i for (i, cpos) in diameters]

In [20]:
subjects = sorted(set(df["participant"]))
positions = sorted(set(df["compound_position"]))
muscles = model.response

In [21]:
stim_max = []

for s in subjects:
    for p in positions:
        for m in muscles: 
            temp = df[(df['participant'] == s) 
                    & (df['compound_position'] == p) 
                    ]
            temp = temp.loc[:,[m, 'pulse_amplitude']]
            temp = temp.pulse_amplitude.max()
            stim_max.append(temp)

stim_max = ma.array(stim_max)
stim_max = stim_max.reshape(len(subjects), len(positions), len(muscles))
            

In [28]:
a = posterior_samples[site.a]
a.shape

(4000, 8, 21, 6)

In [31]:
(stim_max/ma.mean(a, axis=0)).shape

(8, 21, 6)

In [32]:
ma.mean(ma.median((stim_max/ma.mean(a, axis=0)),axis = (1,2)),axis =0)

3.5009097655044012

In [36]:
param.shape

(4000, 8, 21, 6)

In [ ]:
def get_normalized_input_output(subset):
    norm_y = []
    norm_x = []

    for subject_ind, subject in enumerate(subjects):
        curr_params = [
            param[:, subject_ind, :, :][:, :, :, None] for param in params
        ]
        # Take nanmean across posterior samples 
        temp = np.nanmean(curr_params[0], axis=0)
        # Take nanmedian across compound positions and muscles
        median_threshold = np.nanmedian(temp)
        
        x_temp = np.linspace(0.,  median_threshold * MAX_THRESHOLD_CONST, NUM_THRESHOLD_POINTS)
        x_temp = x_temp[None, None, None, :]

        temp_thresh = F.rectified_logistic(
            x_temp,
            *curr_params
        )
        # Remove offset
        temp_thresh = temp_thresh - curr_params[2]
        
        norm_y.append(temp_thresh)
        norm_x.append(x_temp)

    norm_x = np.array(norm_x)
    norm_y = np.array(norm_y)
    norm_y = ma.masked_invalid(norm_y)
    
    return norm_x, norm_y



(8, 4000, 21, 6, 500)
(8, 1, 1, 1, 500)


In [34]:

norm_y = np.swapaxes(norm_y, 1, 2)
norm_x = np.swapaxes(norm_x, 1, 2)
print()
print(norm_y.shape)
print(norm_x.shape)


(8, 21, 4000, 6, 500)
(8, 1, 1, 1, 500)
